In [4]:
import pandas as pd
import numpy as np
import nltk, re
from collections import defaultdict
import pickle

In [1]:
cwd = 'C:/Users/klouc/Desktop/slovcho/src/backend/spell_checking'

sents = pd.read_csv(rf"{cwd}/sents1.csv",encoding='utf-8')
valid_words = pd.read_csv(rf"{cwd}/single_words_bg.csv",encoding='utf-8')

valid_words_dict = defaultdict(int)
for word in list(valid_words['word']):
    # print(word)
    valid_words_dict[word] = word

print("Number of sentences:")
print(len(sents.values))
print("Number of words:")
print(len(valid_words.values))

Number of sentences:
2221299
Number of words:
1147599


In [8]:
# Pre-process sents
# Sents vocab АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЬЮЯабвгдежзийклмнопрстуфхцчшщъьюя1234567890!@#$%^&*()-_=+`~[]{}|;':"\,./<>

# TODO: Lowers first letter of sent but what if first word has multiple capitalised letters
c = 0
def process_sent(sentence):

    sentence = sentence.split('_____')
    tagged_sent = [word_pos.split('///') for word_pos in sentence]

    # replace_with_space_chars = '[!@#$%^&*()-_=+`~{}\|;\[\]\':\"\,./<>]'

    # CLEAN SENTENCES
    # Replace some chars with space
    # sentence = re.sub(replace_with_space_chars, ' ', sentence)
    # # Replace consecutive spaces with single space
    # sentence = re.sub(r'\s+', ' ', sentence)

    # Remove punctuation
    tagged_sent = [word_pos for word_pos in tagged_sent if(word_pos[1] != 'punct')]

    # If sent below 3 tokens do not consider
    if(len(tagged_sent) < 3): return np.nan
    
    # Clean empty words
    tagged_sent = [word_pos for word_pos in tagged_sent if(word_pos[0] != '')]

    # # Remove first charatcer if space
    # if(sentence[0] == ' '): sentence = sentence[1:]
    # Lower letter if non-capitlised word exists in dict
    first_token = tagged_sent[0][0]
    # if(sentence.find(' ')):
    #     first_word = sentence[:sentence.find(' ')]
    # else:
    #     first_word = sentence
    if(ord(first_token[0]) >= ord('А') and ord(first_token[0]) <= ord('Я') and (valid_words_dict[first_token[0].lower() + first_token[1:]] != 0)):
        first_token = first_token[0].lower() + first_token[1:]
        tagged_sent[0][0] = first_token

    # Remove any escape char tokens
    tagged_sent = [word_pos for word_pos in tagged_sent if(word_pos[0] != '\n' and
                                                           word_pos[0] != '\r' and
                                                           word_pos[0] != '\t' and
                                                           word_pos[0] != '\b' and
                                                           word_pos[0] != '\f') and
                                                           word_pos[0][0] != '\\']
    # tagged_sent = "_____".join(["///".join(word_pos) for word_pos in tagged_sent])
    # print(tagged_sent)
    # # ADD BEGINNING AND CLOSING TAG
    # tagged_sent = f'<s>_____{tagged_sent}_____</s>'
    global c
    if(c % 500000 == 0):
        print(c)
    c += 1
    return tagged_sent
    
cleaned_sents = sents["sent"].apply(lambda x : process_sent(x))

# Drop rows with no content
cleaned_sents.dropna(inplace=True)
cleaned_sents.reset_index(inplace=True, drop=True)

cleaned_sents.head(10)

0
500000
1000000
1500000
2000000


0      [[да, Tx], [живее, Vpitf-r3s], [Кралят, Ncmsf]]
1    [[негов, Hmsi], [водещ, Vpitcar-smi], [бе, Vxi...
2    [[Маджурова.Светът, Ncmsf], [стана, Vppif-o3s]...
3    [[Чарлз, Npmsi], [III, Momsi], [беше, Vxitf-t3...
4    [[той, Ppe-os3m], [наследи, Vpptf-o3s], [трона...
5      [[Да, Tx], [живее, Vpitf-r3s], [Кралят, Ncmsf]]
6    [[дигитално, Ansi], [студио, Ncnsi], [със, R],...
7    [[тя, Ppe-os3f], [беше, Vxitf-t3s], [извършена...
8    [[те, Ppe-op3], [са, Vxitf-r3p], [получили, Vp...
9    [[III?Чарлз, Momsi], [III, Momsi], [е, Vxitf-r...
Name: sent, dtype: object

In [9]:
# Create unigrams and bigrams
import tqdm
from random import shuffle

unigrams = nltk.FreqDist()
bigrams = []


shuffle(cleaned_sents)

for tokenised_sent in tqdm.tqdm(cleaned_sents):
    tokens_only = [word_pos[0] for word_pos in tokenised_sent]
    tokens_only.insert(0, '<s>')
    tokens_only.append('</s>')
    # print(tokens_only)

    tokens_frq = nltk.FreqDist(tokens_only)
    tokens_bigrams = nltk.bigrams(tokens_only)

    unigrams.update(tokens_frq)
    bigrams.extend(tokens_bigrams)

# pairs = []
# for trigrams in trigrams:
#     pair = ((trigrams[0], trigrams[1]), trigrams[2])
#     pairs.append(pair)
# trigrams = nltk.ConditionalFreqDist(pairs)
bigrams = nltk.ConditionalFreqDist(bigrams)


100%|██████████| 2204144/2204144 [02:40<00:00, 13733.03it/s]


<bound method ConditionalFreqDist.conditions of <ConditionalFreqDist with 717835 conditions>>


In [27]:
# Save unigram and bigram objects to files
with open('unigrams.pkl', 'wb') as file:
    pickle.dump(unigrams, file)

with open('bigrams.pkl', 'wb') as file:
    pickle.dump(bigrams, file)

In [10]:
from transformers import pipeline

model = pipeline(
    'ner',
    model='rmihaylov/bert-base-ner-theseus-bg',
    tokenizer='rmihaylov/bert-base-ner-theseus-bg',
    device=0,
    revision=None)
output = model('Бил ли си някога във голямата Франция?')
print(output)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[{'entity': 'B-LOC', 'score': 0.99997365, 'index': 7, 'word': '▁Франция', 'start': 29, 'end': 37}]


In [5]:
# TODO: Tokenise input in clever way
# TODO: Dealing with non-cyrillic characters
# TODO: How to deal if first word (capitalised but only cause start of sent) is wrong? As in, discerning capital letter

  # PRE-PROCESS INPUT
def preprocess_input(sent_to_check):
  # Clear punctuation
  sent_to_check = re.sub("[?!.,:]", "", sent_to_check)

  # Tokenise
  words_to_check = sent_to_check.split(' ')

  # Lower first character
  if(ord(words_to_check[0][0]) >= ord('А') and ord(words_to_check[0][0]) <= ord('Я')):
      words_to_check[0] = words_to_check[0].lower()
  # # If beginning word is capitalised and a non-capitalised version exists in the dict, lower it (might be a name)
  # # Else do not lower
  # if(ord(words_to_check[0][0]) >= ord('А') and ord(words_to_check[0][0]) <= ord('Я')):
  #     if(words_to_check[0].lower() in valid_words.values):
  #         words_to_check[0] = words_to_check[0].lower()

  # Add <s> and </s>
  words_to_check.insert(0, '<s>')
  words_to_check.append('</s>')
  return words_to_check

In [16]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'абвгдежзийклмнопрстуфхцчшщъьюя'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def get_word_probabilities(words_to_check, unigram_sum, unigram_count, trust_valid_word_is_correct, bigrams, valid_words_dict, CORRECT_NON_WORDS_ONLY=True):
  sent_probs = []
  for i in range(len(words_to_check)):
      word = words_to_check[i]
      if word != '<s>' and word != '</s>':
          prev_token = words_to_check[i - 1]
          next_token = words_to_check[i + 1]
          candidates = edits1(word)

          # Get candidates which are valid words
          valid_candidates = []
          for candidate in candidates:
              if(valid_words_dict[candidate] != 0):
                  valid_candidates.append(candidate)

          # Non-word errors
          if word not in valid_words.values:
              # Calculate probabilty distribution
              prob_distr = []
              for valid_candidate in valid_candidates:
                  # [MISSING] P(prob of making mistake)

                  # P(candidate) with smooting = probability of word appering
                  # candidate count in word unigram + 1 / count of all words + |V|
                  pc = unigrams[valid_candidate] + 1 / float(unigram_sum + unigram_count)

                  # P(candidate|prev_token) with smoothing = probability of finding candidate followed by prev_token
                  # count of bigrams (prev_token, candidate) + 1 / count of prev_token + |V|
                  pcp = bigrams[prev_token][valid_candidate] + 1 / float(unigrams[prev_token] + unigram_count)

                  # P(next_token|candidate) with smoothing = probability of findging next_token followed by candidate
                  # count of bigrams (candidate, next_token) + 1 / count of candidate + |V|
                  pnc = bigrams[valid_candidate][next_token] + 1 / float(unigrams[valid_candidate] + unigram_count)

                  prob_distr.append(pc * pcp * pnc)
              words_with_probs = [(word, prob) for word, prob in zip(valid_candidates, prob_distr)]
              words_with_probs.sort(key = lambda x: x[1], reverse=True)
              sent_probs.append(words_with_probs)

          # Real world errors
          else:
              if(CORRECT_NON_WORDS_ONLY):
                sent_probs.append([(word, 1)])
                continue
              # Calculate probabilty distribution
              prob_distr = []
              # valid_candidates.append(word)
              for valid_candidate in valid_candidates:
                  # [MISSING] = P(prob of making mistake)

                  # P(candidate) with smooting = probability of word appering
                  # candidate count in word unigram + 1 / count of all words + |V|
                  pc = unigrams[valid_candidate] + 1 / float(unigram_sum + unigram_count)

                  # P(candidate|prev_token) with smoothing = probability of findging candidate followed by prev_token
                  # count of bigrams (prev_token, candidate) + 1 / count of prev_token + |V|
                  pcp = bigrams[prev_token][valid_candidate] + 1 / float(unigrams[prev_token] + unigram_count)

                  # P(next_token|candidate) with smoothing = probability of findging next_token followed by candidate
                  # count of bigrams (candidate, next_token) + 1 / count of candidate + |V|
                  pnc = bigrams[valid_candidate][next_token] + 1 / float(unigrams[valid_candidate] + unigram_count)

                  trust = 0
                  # Apply trust for valid word
                  if(word == valid_candidate):
                      trust = trust_valid_word_is_correct
                  else:
                      trust = 1 - trust_valid_word_is_correct / len(valid_candidate)
                  prob_distr.append(pc * pcp * pnc * trust)
              words_with_probs = [(word, prob) for word, prob in zip(valid_candidates, prob_distr)]
              words_with_probs.sort(key = lambda x: x[1], reverse=True)
              sent_probs.append(words_with_probs)
  return sent_probs


In [13]:
# Load unigram and bigram objects from files
with open('unigrams.pkl', 'rb') as file:
    unigrams = pickle.load(file)

with open('bigrams.pkl', 'rb') as file:
    bigrams = pickle.load(file)

In [17]:
unigram_sum = sum(unigrams.values())
unigram_count = len(unigrams.keys())
trust_valid_word_is_correct = 0.99

cwd = 'C:/Users/klouc/Desktop/slovcho/src/backend/spell_checking'
valid_words = pd.read_csv(rf"{cwd}/single_words_bg.csv",encoding='utf-8')
valid_words_dict = defaultdict(int)
for word in list(valid_words['word']):
    valid_words_dict[word] = word


sent_to_check = 'одавникът умря'
preprocessed_input = preprocess_input(sent_to_check)
probs = get_word_probabilities(preprocessed_input, unigram_sum, unigram_count, trust_valid_word_is_correct, bigrams, valid_words_dict, CORRECT_NON_WORDS_ONLY=True)
for prob in probs:
  print(prob)


[('удавникът', 2.7861449377466165e-06)]
[('умря', 1)]
